**bold text**# Assignment 6: Model Predictive Control I

University of California Berkeley

ME C231A, EE C220B, Experiential Advanced Control I

***

These notes were developed by Roya Firoozi and Francesco Borrelli at UC Berkeley. They are protected by U.S. copyright law and by University policy (https://copyright.universityofcalifornia.edu/resources/ownership-course-materials.html).

If you are enrolled in ME C231A/EE C220B you may take notes and make copies of course materials for your own use. You may also share those materials with another student who is registered and enrolled in this course, and with DSP.

You may not reproduce, distribute or display (post/upload) (Links to an external site.) lecture notes or recordings or course materials in any other way — whether or not a fee is charged — without my express written consent. You also may not allow others to do so. If you do so, you may be subject to student conduct proceedings under the Links to an external site.Berkeley Code of Student Conduct, including Sections 102.23 and 102.25.

***

In this homwowrk assignment, we are going to use $\texttt{Polytope}$ package. For more information please look at this link: https://tulip-control.github.io/polytope/
Note that this package is already installed on docker image. If you are using Google Colab please run the following cell which includes $\texttt{Polytope}$ package installation.  

In [ ]:
# install required dependencies
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
  !pip install polytope
  !pip install -q pyomo
  !apt-get install -y -qq glpk-utils
  !apt-get install -y -qq coinor-cbc
  !wget -N -q "https://ampl.com/dl/open/ipopt/ipopt-linux64.zip"
  !unzip -o -q ipopt-linux64

The following helper functions compute Minkowski sum, precursor set and $\mathcal{O}_{\infty}$ set. Take a look at the code. You are going to use them in this howework. 

In [ ]:
import polytope as pt
import numpy as np
import matplotlib.pyplot as plt
#%matplotlib widget

# Helper Functions:

def minkowski_sum(X, Y):

    # Minkowski sum between two polytopes based on 
    # vertex enumeration. So, it's not fast for the
    # high dimensional polytopes with lots of vertices.
    V_sum = []
    if isinstance(X, pt.Polytope):
        V1 = pt.extreme(X)
    else:
        # assuming vertices are in (N x d) shape. N # of vertices, d dimension
        V1 = X
        
    if isinstance(Y, pt.Polytope):
        V2 = pt.extreme(Y)
    else:
        V2 = Y

    for i in range(V1.shape[0]):
        for j in range(V2.shape[0]):
            V_sum.append(V1[i,:] + V2[j,:])
    return pt.qhull(np.asarray(V_sum))


def precursor(Xset, A, Uset=pt.Polytope(), B=np.array([]), Wset=pt.Polytope()):
    
    if not B.any():
        if not pt.is_empty(Wset):
            return pt.Polytope( pontryagin_diff(Xset, Wset).A @ A, Xset.b)
        else:
            return pt.Polytope(Xset.A @ A, Xset.b)
        
    if not pt.is_empty(Wset):
        tmp  = minkowski_sum( pontryagin_diff(Xset, Wset), pt.extreme(Uset) @ -B.T )
        return pt.Polytope(tmp.A @ A, tmp.b)
    else:
        tmp  = minkowski_sum( Xset, pt.extreme(Uset) @ -B.T )
        return pt.Polytope(tmp.A @ A, tmp.b)
    
    
    
def Oinf(Xset, A, Wset=pt.Polytope()):
    
    Omega = Xset
    k = 0
    Omegap = precursor(Omega, A).intersect(Omega)
    while not Omegap == Omega:
        k += 1
        Omega = Omegap
        if not pt.is_empty(Wset):
            Omegap = pt.reduce(precursor(Omega, A, Wset=Wset).intersect(Omega))
        else:
            Omegap = pt.reduce(precursor(Omega, A).intersect(Omega))
    return Omegap


***

# <font color=blue> 1. Formulation of CFTOC Problem with Pyomo</font>

In this problem, you will write a function which formulates and solves a finite time optimal control problem with quadratic cost, linear dynamics, and linear constraints. You will be using this function in the next part of the homework as a part of your MPC controller. Consider the problem defined below:
\begin{equation}
\begin{aligned}
\min_{x_1 \dots x_N,u_0 \dots u_{N-1}} & \quad x_N^{\top}Px_N+\sum_{k=0}^{N-1}x_k^{\top}Qx_k+u_k^{\top}Ru_k \\
\text{s.t.} & \quad x_{k+1} = Ax_k + Bu_k \quad \forall k=0\dots N-1\\
& \quad x_{min} \leq x_k \leq x_{max}  \quad \forall k=0\dots N-1 \\
& \quad u_{min} \leq u_k \leq u_{max}  \quad \forall k=0\dots N-1 \\
& \quad x_0 = x(0) \\
& \quad x_N \in \mathcal{X}_f
\end{aligned}
\end{equation}

Write a function in the following form which formulates the problem using pyomo and solves the problem using the solver $\texttt{ipopt}$. Test your code on several 1 and 2-state examples.

`def solve_cftoc(A, B, P, Q, R, N, x0, xL, xU, uL, uU, bf, Af):
    return [feas, xOpt, uOpt, JOpt]`


- $\texttt{feas}$ is a scalar logical indicating the feasibility of the CFTOC problem
- $\texttt{xOpt}$ is a $n_x \times (N+1)$ matrix of the optimal state trajectory, where $n_x$ is the state dimension
- $\texttt{uOpt}$ is a $n_u \times N$ matrix of the optimal input sequence, where $n_u$ is the input dimension
- $\texttt{JOpt}$ is a scalar of the optimal cost from the specified initial condition
- $\texttt{A, B}$ are the state and input matricies of the linear dynamics
- $\texttt{P}$ is the terminal cost matrix
- $\texttt{Q, R}$ are the stage cost matricies
- $\texttt{N}$ is the MPC horizon
- $\texttt{x0}$ is the initial condition
- $\texttt{xL, xU, uL, uU}$ are the box state and input constraints
- $\texttt{bf, Af}$ are used to represent the terminal set. If the terminal set ${\cal X}_f$ is a single point, ${\cal X}_f = \{ \bar{x}_N \}$, the input argument $\texttt{bf}$ is used to specify this single point $\bar{x}_N$, and the input argument $\texttt{Af = []}$ is used to designate this special case of a single-point terminal set. If the terminal set is described by a set of linear inequalities, it can be expressed using the inequality $A_f x_N \leq b_f$.


### <font color=red> Delivarable 1: write your answer in the code cell below.</font>

In [ ]:
from __future__ import division
import pyomo.environ as pyo
import numpy as np
import polytope as pt

# Write your code here
def solve_cftoc(A, B, P, Q, R, N, x0, x1U, x2U, uL, uU, bf, Af):
    
    model = pyo.ConcreteModel()
    model.N = N
    model.nx = np.size(A, 0)
    model.nu = np.size(B, 1)
    model.nf = np.size(Af, 0)
    
    # length of finite optimization problem:
    model.tIDX = pyo.Set( initialize= range(model.N+1), ordered=True )  
    model.xIDX = pyo.Set( initialize= range(model.nx), ordered=True )
    model.uIDX = pyo.Set( initialize= range(model.nu), ordered=True )
    
    model.nfIDX = pyo.Set( initialize= range(model.nf), ordered=True )
    
    # these are 2d arrays:
    model.A = A
    model.B = B
    model.Q = Q
    model.P = P
    model.R = R
    model.Af = Af
    model.bf = bf
    
    # Create state and input variables trajectory:
    model.x = pyo.Var(model.xIDX, model.tIDX)
    model.u = pyo.Var(model.uIDX, model.tIDX, bounds=(uL,uU))
    
    #Objective:
    def objective_rule(model):
        costX = 0.0
        costU = 0.0
        costTerminal = 0.0
        for t in model.tIDX:
            for i in model.xIDX:
                for j in model.xIDX:
                    if t < model.N:
                        costX += model.x[i, t] * model.Q[i, j] * model.x[j, t] 
        for t in model.tIDX:
            for i in model.uIDX:
                for j in model.uIDX:
                    if t < model.N:
                        costU += model.u[i, t] * model.R[i, j] * model.u[j, t]
        for i in model.xIDX:
            for j in model.xIDX:               
                costTerminal += model.x[i, model.N] * model.P[i, j] * model.x[j, model.N]
        return costX + costU + costTerminal
    
    model.cost = pyo.Objective(rule = objective_rule, sense = pyo.minimize)
    
    # Constraints:
    def equality_const_rule(model, i, t):
        return model.x[i, t+1] - (sum(model.A[i, j] * model.x[j, t] for j in model.xIDX)
                               +  sum(model.B[i, j] * model.u[j, t] for j in model.uIDX) ) == 0.0 if t < model.N else pyo.Constraint.Skip
    

    model.equality_constraints = pyo.Constraint(model.xIDX, model.tIDX, rule=equality_const_rule)
    model.init_const1 = pyo.Constraint(expr = model.x[0, 0] == x0[0])
    model.init_const2 = pyo.Constraint(expr = model.x[1, 0] == x0[1])
    model.state_limit1 = pyo.Constraint(model.tIDX, rule=lambda model, t: model.x[0, t] <= x1U
                                   if t < N else pyo.Constraint.Skip)
    model.state_limit2 = pyo.Constraint(model.tIDX, rule=lambda model, t: -x1U <= model.x[0, t]
                                    if t < N else pyo.Constraint.Skip)
    model.state_limit3 = pyo.Constraint(model.tIDX, rule=lambda model, t: model.x[1, t] <= x2U
                                   if t < N else pyo.Constraint.Skip)
    model.state_limit4 = pyo.Constraint(model.tIDX, rule=lambda model, t: -x2U <= model.x[1, t]
                                    if t < N else pyo.Constraint.Skip)
    if len(Af) == 0:
      model.terminal_cost1 = pyo.Constraint(expr = model.x[0, model.N] == bf)
      model.terminal_cost1 = pyo.Constraint(expr = model.x[1, model.N] == bf)
    else:
      def final_const_rule(model, i):
        return sum(model.Af[i, j] * model.x[j, model.N] for j in model.xIDX) <= model.bf[i] 
      model.terminal_constraints = pyo.Constraint(model.xIDX, rule = final_const_rule)
    
    model.final_const = pyo.Constraint(model.nfIDX, rule=final_const_rule)
    
    solver = pyo.SolverFactory('ipopt')
    results = solver.solve(model)
    
    if str(results.solver.termination_condition) == "optimal":
        feas = True
    else:
        feas = False
            
    xOpt = np.asarray([[model.x[i,t]() for i in model.xIDX] for t in model.tIDX]).T
    uOpt = np.asarray([model.u[:,t]() for t in model.tIDX]).T
    
    JOpt = model.cost()
      
    return [feas, xOpt, uOpt, JOpt]

[feas, xOpt, uOpt, JOpt] = solve_cftoc(A, B, P, Q, R, N, x0, x1U, x2U, -uU, uU, bf, Af)

IndentationError: ignored

In this problem, we will design an MPC controllers of the form: 

\begin{equation*}
   \begin{array}{lll}
       J_0^*(x_0)=  &\displaystyle{\min_{{U_0}}}& \displaystyle{ \color{red}{x_N'Px_N} + \sum_{k=0}^{{N}-1}
  x_k'Qx_k+u_k'Ru_k} \\
       &\text{subject to}   \\
       &&x_{k+1}=Ax_{k}+Bu_{k},\ k=0,\ldots,N-1\\
                       &&x_{k}\in\mathcal{X},~u_{k}\in\mathcal{U},~k=0,\ldots,N-1\\
                       && \color{red}{{x_{N}\in {\mathcal{X}_f} }} \\
                       && x_{0}=x(t)
    \end{array}
\end{equation*}
In this problem, $Q=I_2$, $R=10$, $P=Q$ and $N=3$.

In [ ]:




print('JOpt=', JOpt)
print('xOpt=', xOpt)
fig = plt.figure(figsize=(9, 6))
plt.plot(xOpt.T)
plt.ylabel('x')
fig = plt.figure(figsize=(9, 6))
plt.plot(uOpt.T)
plt.ylabel('u')
plt.show()

***

# <font color=blue> 2. Persistently Feasible and Asymptotically Stable Linear MPC design </font>


Consider the following system
\begin{equation}\label{eq:model-int1}
x(k+1)=\begin{bmatrix} 1.2 &  1\\ 0 & 1 \end{bmatrix}x(k)+
     \begin{bmatrix} 0\\ 1 \end{bmatrix}u(k) \quad \quad (1)   
\end{equation}

The state and input constraints are
\begin{equation}\begin{aligned}
\mathcal{U}: &-1\leq u(k)\leq 1 \quad \quad \quad \quad \quad \quad \quad \quad \quad (2a)\\ 
\mathcal{X}: &\begin{bmatrix} -15\\ -15 \end{bmatrix} \leq x(k) \leq \begin{bmatrix} 15\\ 15 \end{bmatrix} \quad \quad \quad \quad \quad \quad (2b)
\end{aligned}
\label{eq:con1}\end{equation}

In this problem, we will design several MPC controllers of the form: 

\begin{equation*}
   \begin{array}{lll}
       J_0^*(x_0)=  &\displaystyle{\min_{{U_0}}}& \displaystyle{ \color{red}{x_N'Px_N} + \sum_{k=0}^{{N}-1}
  x_k'Qx_k+u_k'Ru_k} \\
       &\text{subject to}   \\
       &&x_{k+1}=Ax_{k}+Bu_{k},\ k=0,\ldots,N-1\\
                       &&x_{k}\in\mathcal{X},~u_{k}\in\mathcal{U},~k=0,\ldots,N-1\\
                       && \color{red}{{x_{N}\in {\mathcal{X}_f} }} \\
                       && x_{0}=x(t)
    \end{array}
\end{equation*}
In this problem, we will fix  the dynamics ($A$, $B$), the constraint sets $\mathcal{X}$ and $\mathcal{U}$, $Q=I_2$, $R=1$, and $N=3$ and only examine how the choice of $P$ and $\mathcal{X}_f$ affect the set of feasible initial states $\mathcal{X}_0$.
$\textit{Note}$: Much of this exercise refers to the MPC and Reachability slides. Make sure you understand those slides before proceeding. Make sure you downloaded the latest sets of slides.



# <font color=blue> Part a) Approach 1 <font>

Choose $\mathcal{X}_f=\{0\}$ (note that $x_N'Px_N=0$ for any $P$ since $x_N=0$). Write code to formulate and solve the MPC problem over a simulation horizon $M$ from initial conditions $x_0$. Print out $\texttt{feas}$, $\texttt{xOpt}$, $\texttt{uOpt}$, and $\texttt{predErr}$ arrays.

$\texttt{feas}$ is a $1 \times M$ vector of logicals that indicate the feasibility of the problem at each simulation step (e.g. if the problem is feasible for 3 steps, then $\texttt{feas}=[\texttt{true true true false} ... \texttt{false}])$ . If the problem becomes infeasible at any time, you should return empty arrays for $\texttt{xOpt}$, $\texttt{uOpt}$, and $\texttt{predErr}$.
$\texttt{xOpt}$ is an array of the optimal closed-loop trajectory of size $2 \times (M+1)$
$\texttt{uOpt}$ is an array of the optimal input sequence of size $1 \times M$
$\texttt{predErr}$ is a $2 \times (M-N+1)$ array of the $l_2$-norm of the difference between the open-loop predictions and the closed-loop trajectory for each state from simulation timestep 0 to $M-N$. To clarify further, the difference between the open-loop predictions and the closed-loop trajectory is a vector at each time, you need to take the $l_2$-norm of this vector to compute an element of the $\texttt{predErr}$ array.


You should call the function $\texttt{solve_cftoc}$, that you wrote in Problem 1, in the simulation time loop.

1.  Let $x(0) = [2,-1]^T$. Plot on the same figure the closed-loop state trajectory (plot $x(2)$ vs $x(1)$ as a solid line) as well as the open-loop trajectories (dashed lines) predicted by the MPC with a simulation horizon of $M=25$.

2. For the same $x(0) = [2,-1]^T$, analyze the mismatch between predicted vs closed-loop trajectories using the $\texttt{predErr}$ output as you increase $N$ in the MPC design. Show with plots how this prediction error changes with $N$.

3. Plot the set of feasible initial states $\mathcal{X}_0$. Note: You have two options by using brute-force gridding or by using polyhedral projections.

4. Is $\mathcal{X}_0$ an invariant set for the dynamic system (1) subject to the constraints (2) under the MPC law you designed? Justify your answer. (Hint: look at the Reachability slides for the definition of an invariant set.)

5. Is the MPC you just designed persistently feasible for all initial states in $\mathcal{X}_0$?  

6. Is the origin asymptotically stable for the closed loop system? 

7.  What is its region of  attraction? (Hint: look at the MPC slides)

### <font color=red> Delivarable 2a: write your answer in the two code cells below.</font>

In [ ]:
import numpy as np
import polytope as pt

# Write your code here:
A = np.array([[1.2, 1],
               [0, 1]])
B = np.array([[0],
              [1]])


Q = np.eye(2)
R = np.array([1]).reshape(1,1)
P = Q
N = 3
xL = -15
uL = -1
xU = 15
uU = 1
x0 = np.array([2, -1])



In [ ]:
nx = np.size(A, 0)
nu = np.size(B, 1)

M = 25   # Simulation time

xOpt = np.zeros((nx, M+1))
uOpt = np.zeros((nu, M))
xOpt[:, 0] = x0.reshape(nx, )

xPred = np.zeros((nx, N+1, M))

feas = np.zeros((M, ), dtype=bool)

fig = plt.figure(figsize=(9, 6))
for t in range(M):
    [model, feas[t], x, u, J] = solve_cftoc(A, B, P, Q, R, N, xOpt[:, t], xL, xU, uL, uU)
    
    if not feas[t]:
        xOpt = []
        uOpt = []
        break
        

         # Save open loop predictions
    xPred[:, :, t] = x
        
    xOpt[:, t+1] = x[:, 1]
    uOpt[:, t] = u[:, 0].reshape(nu, )

    line1 = plt.plot(x[0,:], x[1,:], 'r--')



print('Feasibility =', feas)     

line1 = plt.plot(xOpt[0, :], xOpt[1, :], 'bo-')
plt.legend([line1[0], line2[0]], ['Open-loop', 'Closed-loop']);

plt.xlabel('x1')
plt.ylabel('x2')
plt.axis('equal')

In [ ]:
xU = 15
uU = 1
[model, feas, xOpt, uOpt, JOpt] = solve_cftoc(A, B, P, Q, R, N, x0, xL, xU, uL, uU)


# Find the prediction error
for t in range(len(predErr)):
    # xPred: open-loop trajectory
    err = xOpt[:, t:t+N+1] - xPred[:,:, t]
    predErr[:, t] = np.array([np.linalg.norm(err[0,:]), np.linalg.norm(err[1,:])]).reshape(nx, )    

# Print predErr
print(predErr)

# constraint sets represented as polyhedra
X = pt.Polytope(np.array([[1.0, 0], 
                          [0, 1.0],
                          [-1, 0],
                          [0, -1]]), 
                np.array([[xU], 
                          [xU],
                          [xU],
                          [xU]]))

U = pt.Polytope(np.array([1, -1]).reshape(2,1),
                np.array([uU, uU]).reshape(2,1))


# target set

# Since the Polytope package cannot handle non full-dimensional
# polytopes, we need to define an epsilon box around zero in
# order to run the code:
eps = 0.001
S = pt.box2poly([[-eps, eps], [-eps, eps]])  # the target set S is Xf = {0}

# Write your code here:
# N-step Controllable Set

N = 3

C = {}
PreS = precursor(S, A, U, B)
for j in range(N):
    C[j]= PreS.intersect(X)
    PreS = precursor(C[j], A, U, B)

X0 = C[N-1]  # The initial feasible set X0 is equivalent to the N-step controllable set. 


# Plotting the polytopic sets and the closed loop trajectory 
plt.clf()
plt.cla()
plt.close('all')

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)

X.plot(ax, color='m', alpha=0.1, linestyle='solid', linewidth=1, edgecolor=None)      # state constraint set
# C_inf.plot(ax, color='m', alpha=0.6, linestyle='solid', linewidth=1)    # maximal control invariant set 
X0.plot(ax, color='y', alpha=0.7, linestyle='solid', linewidth=1)       # initial feasible set 
# O_inf.plot(ax, color='g', alpha=0.7, linestyle='solid', linewidth=1)    # maximal positive invariant set 
# ax.legend(['X', 'Cinf', 'X0', 'Oinf'])

ax.plot(xOpt[0, :], xOpt[1, :], 'bo-', markerfacecolor='none', markeredgewidth=0.5, linewidth= 0.5, label='traj')  # closed loop trajectory 
ax.plot(xOpt[0, 0], xOpt[1, 0], 'ro', label='init') # the initial condition x0

***

# <font color=blue> Part b) Approach 2 </font>

To design our MPC, now use a stabilizing controller $u = Kx$ where $K$ is:
\begin{align*}
K &= -\begin{bmatrix}  1.595  &  2.35 \end{bmatrix}
\end{align*}
which  produces a stable closed loop system with poles at 0.1 and -0.25. Let $\color{red}{{X}_f}$ be the maximal invariant set $O_\infty$ for a closed loop system $x(k+1) = (A+BK)x(k)$ subject to
constraints (3).
Find a new terminal cost such that it is a Lyapunov Function for $x \in \color{red}{{X}_f} = O_\infty$ (note: you should be able to design a quadratic terminal cost with weight $P$ by simply using $\texttt{scipy.linalg.solve_discrete_lyapunov}$). Write the code to formulate and solve the MPC problem. (Note: Your MPC code will be largely similar to your previous one, with the exception that you are now using different terminal constraints). Print out $\texttt{feas}$, $\texttt{xOpt}$, $\texttt{uOpt}$, and $\texttt{predErr}$ arrays.

$\texttt{feas}$ is a $1 \times M$ vector of logicals that indicate the feasibility of the problem at each simulation step (e.g. if the problem is feasible for 3 steps, then $\texttt{feas}=[\texttt{true true true false} ... \texttt{false}]$ . If the problem becomes infeasible at any time, you should return empty arrays for $\texttt{xOpt}$, $\texttt{uOpt}$, and $\texttt{predErr}$.
$\texttt{xOpt}$ is an array of the optimal closed-loop trajectory of size $2 \times (M+1)$
$\texttt{uOpt}$ is an array of the optimal input sequence of size $1 \times M$
$\texttt{predErr}$ is a $2 \times (M-N+1)$ array of the $l_2$-norm of the difference between the open-loop predictions and the closed-loop trajectory for each state from simulation timestep 0 to $M-N$. To clarify further, the difference between the open-loop predictions and the closed-loop trajectory is a vector at each time, you need to take the $l_2$-norm of this vector to compute an element of the $\texttt{predErr}$ array.

You should call the function $\texttt{solve_cftoc}$, that you wrote in Problem 1, in the simulation time loop.

1.  Let $x(0) = [2,-1]^T$. Plot on the same figure the closed-loop state trajectory (plot $x(2)$ vs $x(1)$ as a solid line) as well as the open-loop trajectories (dashed lines) predicted by the MPC with a simulation horizon of $M=25$.

2. For the same $x(0) = [2,-1]^T$, analyze the mismatch between predicted vs closed-loop trajectories using the $\texttt{predErr}$ output as you increase $N$ in the MPC design. Show with plots how this prediction error changes with $N$.

3. Plot the set of feasible initial states $\mathcal{X}_0$. Note: You have two options by using brute-force gridding or by using polyhedral projections.

4. Is $\mathcal{X}_0$ an invariant set for the dynamic system (1) subject to the constraints (2) under the MPC law you designed? Justify your answer. (Hint: look at the Reachability slides for the definition of an invariant set and the MPC slides.)

5. Is the MPC you just designed persistently feasible for all initial states in $\mathcal{X}_0$? 

6. Is the origin asymptotically stable for the closed loop system? 

7.  What is its region of attraction?

### <font color=red> Delivarable 2b: write your answer in the three code cells below.</font>

In [ ]:
import scipy.linalg
import numpy as np
import polytope as pt

# Fill in the blanks: 

K = ...
A_cl = ...   # closed-loop A matrix
P_2 = scipy.linalg.solve_discrete_lyapunov(...)    # The weight for terminal cost
S = X.intersect(...)           
O_inf = Oinf(...)
Af = O_inf.A
bf = O_inf.b






# Write your code here:


***

# <font color=blue> Part c) Approach 3 </font>
Now set $\mathcal{X}_f$ to be the maximal invariant set $O_\infty$ for the closed loop system $x(k+1) = (A+BF_\infty)x(k)$ subject to
the given constraints. Here $F_\infty$ is the infinite time LQR controller for the system $x(k+1) = Ax(k)+Bu(k)$ with weights $Q$ and $R$ defined previously, such that $u(k)=F_{\infty}x(k)$. Choose the terminal cost $P$ to be the infinite time cost associated with the controller $F_\infty$. (Hint: Use $\texttt{scipy.linalg.solve_discrete_are}$). Write your code to solve the MPC problem. (Note: your code will be largely similar to your previous part, with the exception that you are now using different terminal constraints)

1. Plot $O_\infty$ in this approach  with the one in Approach 2 and discuss the differences.

2. Let $x(0) = [2,-1]^T$ and plot the closed-loop state trajectory as well as the open-loop trajectories predicted by the MPC with this new terminal cost and terminal constraint for 25 simulation steps ($N$ is still equal to 3). How does this compare to the other solutions?

3. Plot the set of  feasible initial states $\mathcal{X}_0$. Note: You have two options by using brute-force gridding or by using polyhedral projections.

4. Is $\mathcal{X}_0$ an invariant set for the dynamic system (1) subject to the constraints (2) under the MPC law you designed? Justify your answer. 


5. Is the MPC you just designed persistently feasible for all initial states in $\mathcal{X}_0$? 


6. Is the origin asymptotically stable for the closed loop system? 


7. What is its region of  attraction? 

### <font color=red> Delivarable 2c: write your answer in the two code cells below.</font>

In [ ]:
# Fill in the blanks and write your code

import scipy.linalg

def dlqr(A, B, Q, R):
    
    # Write your code here:

    return F_inf, P_inf, eigVals

F_inf, P_inf, eigvals = dlqr(A, B, Q, R)
A_cl = ...
S = X.intersect(...)
O_inf = ...
Af = ...
bf = ...


# Write your code here:


***

# <font color=blue> Part d) Comparison </font>
Plot and compare $\mathcal{X}_0$ for the three MPC designs (Approach 1,2, and 3) for $N=3$. What do you notice?

### <font color=red> Delivarable 2d: write your answer in the the code cells below.</font>

In [ ]:
# Write your code here: 